In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy import create_engine, VARCHAR
from sqlalchemy import Table, MetaData, Column
from tqdm import tqdm

### Identifiants

In [ ]:
# Informations d'identification PostgreSQL
db_user = "terence"
db_password = "Canon0-Spinout8-Debatable6-Proofread2"
db_host = "stockage-donnee-foncieres-gr02.postgres.database.azure.com"
db_port = 5432
db_name = "postgres"

In [ ]:
# Créez une connexion à la base de données
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [2]:
# Chargez le fichier CSV dans un DataFrame Pandas
df = pd.read_csv('./src/datasets/csv/valeurs_foncieres_affichage.csv', encoding='utf-8', dtype={'code_commune': str, 'code_departement': str})


In [3]:
df.head()

,Unnamed: 0,date_mutation,valeur_fonciere,code_commune,code_departement,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain
0,0,2018-07-02,360000.0,1374,1,Maison,170.0,7.0,3789.0
1,1,2018-07-02,360000.0,1374,1,Maison,170.0,7.0,500.0
2,2,2018-07-02,360000.0,1374,1,Maison,170.0,7.0,1500.0
3,3,2018-07-05,190000.0,1304,1,Maison,90.0,4.0,903.0
4,4,2018-07-04,220000.0,1314,1,Maison,99.0,3.0,818.0


In [ ]:
# Nom de la table dans la base de données
table_name = 'valeur_fonciere'

In [ ]:
# Supprimer la colonne inutile "Unnamed: 0"
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Supprimer la table existante
with engine.connect() as connection:
    # Supprimer la table si elle existe déjà
    drop_table_query = text("DROP TABLE IF EXISTS valeur_fonciere")
    connection.execute(drop_table_query)

    # Créer la table en fonction de l'en-tête du fichier CSV
    metadata = MetaData()
    table = Table(table_name, metadata, *[Column(col, VARCHAR) for col in df.columns])
    metadata.create_all(connection)

    # Nombre total de lignes dans le DataFrame
    total_rows = len(df)

    # Barre de progression tqdm
    with tqdm(total=total_rows, desc="Insertion des données dans la base de données") as pbar:
        # Itérer sur les lignes du DataFrame
        for index, row in df.iterrows():
            # Insérer la ligne dans la table
            connection.execute(table.insert().values(**row.to_dict()))

            # Mettre à jour la barre de progression
            pbar.update(1)

print("Les données ont été insérées avec succès.")